In [46]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import r2_score


In [47]:
# load the data, prepare it for sklearn
data = pd.read_excel("bodyfat.xlsx")
m = data.to_numpy()
X, y = m[:, 1:], m[:, 0]

labels = data.columns.values[1:]

In [48]:
print(labels)

['age [yr]' 'weight [kg]' 'height [cm]' 'bmi' 'neck [cm]' 'chest [cm]'
 'abdomen [cm]' 'hip [cm]' 'tight [cm]' 'knee [cm]' 'ankle [cm]'
 'biceps [cm]' 'forearm [cm]' 'wrist [cm]']


In [49]:

# normalization
# think about why it has no effect with plain (unregularized) linear regression
# and why the cross-validated accuracy changes between raw and normalized data
# when we use regularization
#X = X - X.mean(axis=0)
#X /= X.std(axis=0)

# selection of the model
# model_type = LinearRegression
model_type, model_name = \
    [(Ridge, "Ridge Regression"), (Lasso, "Lasso Regression")][1]

# cross validation
# for the start, to figure out if we are doing well
# and to compare between normalized and raw data
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = LinearRegression()
y_hat = cross_val_predict(model, X, y, cv=cv, n_jobs=-1)
r2 = r2_score(y, y_hat)
print(f"R2 for unregularized regression: {r2:.3}")

R2 for unregularized regression: 0.676


In [50]:
# choice of regularization parameters
alphas = np.logspace(-4, 5, 20)

# how does the accuracy change with regularization strength?
# we cross-validate to assess the accuracy and plot
# the accuracy(regularization strength) graph
r2s = []
for alpha in alphas:
    model = model_type(alpha=alpha)
    y_hat = cross_val_predict(model, X, y, cv=cv, n_jobs=-1)
    r2 = r2_score(y, y_hat)
    r2s.append(r2)

index_max = np.argmax(r2s)
best_alpha = alphas[index_max]
print(np.max(r2s))
print(f"Best regularization strength: {best_alpha:.2}")

/home/gganbu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.753e+01, tolerance: 1.296e+00
  model = cd_fast.enet_coordinate_descent(
/home/gganbu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.003e+01, tolerance: 1.296e+00
  model = cd_fast.enet_coordinate_descent(


0.7117081252796834
Best regularization strength: 0.21


In [23]:
fig = plt.figure()
ax = plt.gca()
ax.plot(alphas, r2s, "o-")
ax.set_xscale('log')
plt.xlabel("regularization strength")
plt.ylabel("cross validated r2")
plt.savefig("fig-accuracy-vs-regularization.pdf")
plt.clf()

<Figure size 432x288 with 0 Axes>

In [24]:
# select best-ranked features for specific degree of regularization
alpha = 0.1
model = model_type(alpha=alpha)
fitted = model.fit(X, y)
coef = np.abs(fitted.coef_)

k = 5 # number of best-rank features to select
ind = np.argpartition(coef, -k)[-k:]

# compute coefficients for the regularization path
cs = []
for alpha in alphas:
    model = model_type(alpha=alpha)
    fitted = model.fit(X, y)
    cs.append(fitted.coef_)
res = np.stack(cs)

# plot the regularization path for selected features
fig = plt.figure()
ax = plt.gca()
for i in ind:
    ax.plot(alphas, res[:, i], "o-", label=labels[i])
ax.legend(loc="upper right")
ax.set_xscale('log')
plt.xlabel("regularization strength")
plt.ylabel("feature weight")
plt.savefig("fig-regularization-path.pdf")
plt.clf()

<Figure size 432x288 with 0 Axes>